In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.distances import distance_array

import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
import random
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

import torch
from torch import nn, optim

/home/eurastof/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Making coordinate data (b2ar)


dirs = glob.glob("../binding_spots_project/gpcr_sampling/b2ar/b2ar_centered_aligned/*")
data = []
ids = []
for d in dirs:
    print(d)
    system = d.split("/")[-1]
    cosmos = mda.Universe(d + "/step6.6_equilibration.gro", glob.glob(f"{d}/*xtc"))
    heavy = cosmos.select_atoms("protein and not type H")
    coords = []
    for ts in cosmos.trajectory[0:-1:2]:
        pos = heavy.positions.flatten()
        coords.append(pos.reshape(1, pos.shape[0]))
    coords = np.concatenate(coords, axis=0)
    ids += [system for _ in range(coords.shape[0])]
    data.append(coords)

data = np.concatenate(data)


cols = []
for resid, i in enumerate(range(0, data.shape[1], 3)):
    cols += [f"{resid}x", f"{resid}y", f"{resid}z"]

D = pd.DataFrame(data, columns=cols)
D["system"] = ids


D.to_csv("./data/b2ar-heavy-atoms.csv")

## Autoencoder

In [2]:

class Autoencoder(BaseEstimator, TransformerMixin, nn.Module):

    def __init__(self, in_shape=10, enc_shape=2, middle_shape=5, n_hidden=1, loss_fn=nn.L1Loss(), lr=1e-3):
        
        super().__init__()
        self.loss_fn = loss_fn
        self.lr = lr 
        self.n_hidden = n_hidden # number of hidden layers
        self.in_shape = in_shape # input dimension
        self.enc_shape = enc_shape # dimension of encoding
        self.middle_shape = middle_shape # hidden layer dimensions
        
        encoder_layers = [nn.Linear(self.in_shape, self.middle_shape), nn.ReLU(), nn.Dropout(0.2)] # initialize encoder layer list
        decoder_layers = [nn.Linear(self.enc_shape, self.middle_shape), nn.ReLU(), nn.Dropout(0.2)] # initialize decoder layer list

        for i in range(n_hidden - 1): # Add layers to encoder and decoder according to n_hidden and middle shape
            encoder_layers.append(nn.Linear(self.middle_shape, self.middle_shape))
            encoder_layers.append(nn.ReLU())
            encoder_layers.append(nn.Dropout(0.2))
            decoder_layers.append(nn.Linear(self.middle_shape, self.middle_shape))
            decoder_layers.append(nn.ReLU())
            decoder_layers.append(nn.Dropout(0.2))
            
        encoder_layers.append(nn.Linear(self.middle_shape, self.enc_shape)) # Final encoder layer
        decoder_layers.append(nn.Linear(self.middle_shape, self.in_shape)) # Final decoder layer
        #decoder_layers.append(nn.Sigmoid()) # Sigmoid at end of deocder?

        self.encode = nn.Sequential(*encoder_layers) # Make encoder
        self.decode = nn.Sequential(*decoder_layers) # Make decoder
        

    def fit(self, X, y=None, n_epochs=20, batch_size=32, verbose=False):
        self.training = True # Enables e.g. dropouts to work
        X = torch.Tensor(X)
        indices = [i for i in range(X.shape[0])]
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr) # Adam only atm
        
        for epoch in range(n_epochs):
        
            random.shuffle(indices) # random shuffle to get random batches for each epoch
            batches = [i for i in range(0, len(indices), batch_size)]

            for i in range(len(batches) - 1):

                batch_X = X[indices[batches[i]:batches[i+1]]]
                self.optimizer.zero_grad() # reset optimizer
                
                encoded = self.encode(batch_X)
                decoded = self.decode(encoded)

                loss = self.loss_fn(decoded, batch_X)
                loss.backward() # Backpropagate
                self.optimizer.step() # Apply changes
            
            if verbose:
                print(f'epoch {epoch} \t Loss: {loss.item():.4g}')
        
        return self

    def transform(self, X, y=None):
        encoded = self.encode(torch.Tensor(X))
        return encoded.cpu().detach().numpy()
    

    def inverse_transform(self, X, y=None):
        decoded = self.decode(torch.Tensor(X))
        return decoded.cpu().detach().numpy()
    
    def score(self, X, y=None):
        encoded = self.transform(X)
        decoded = self.inverse_transform(encoded)
        
        return -self.loss_fn(torch.Tensor(X), torch.Tensor(decoded)) # Take negative to make GridSearchCV work properly
        
    


## Autoencoder testing

In [3]:
#device = ('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"  # GridSearchCV having issues with GPUs
print(f"Using {device} device.")

Using cpu device.


## Pipeline

In [33]:
# Grid of parameters for gridsearch

param_grid = {
    "Autoencoder":{
        "Autoencoder__middle_shape": [512],
        "Autoencoder__enc_shape": [2],
        "Autoencoder__n_hidden": [1]
    },
    "GMM": {
        "n_components": [4]
    }
}


In [34]:
# Function to search parameters and return best model

def best_pipeline(transformer, param_grid, X, cv=2):
    
    step_names = ["Scaler", "Autoencoder"]
    params = {key: val for k, d in param_grid.items() for key, val in d.items() if k in step_names}
    pipe = Pipeline(
        steps=[
            (step_names[0], StandardScaler()),
            (step_names[1], transformer),
        ]   
    )
    
    gridsearch = GridSearchCV(pipe, param_grid=params, verbose=3, cv=cv)
    gridsearch.fit(X)
    
    return gridsearch.best_estimator_
    

## Pipeline testing

# Test set

encoded = best.transform(test_X)
decoded = best.inverse_transform(torch.Tensor(encoded))

# L1 loss (Å)

error = np.mean(np.abs(decoded- test_X))
print(f"Reconstruction error: {error} Å ")

## Input perturbation analysis

In [33]:
from sklearn.utils import shuffle

In [5]:

def IPA(X, model):
        
    
    index = np.arange(0, X.shape[1], 3)
    effects = []
    
    for i in index:
        shuffled = X.copy()
        shuffled = shuffle(shuffled)
        rands = np.random.uniform(low=-5, high=5, size=(shuffled.shape[0],1))
        rands = np.concatenate([rands, rands, rands], axis=1)
        shuffled[:, i:i+3] = rands
        encoded = model.transform(shuffled)
        decoded = model.inverse_transform(encoded)
        decoded = s.inverse_transform(decoded)
        L1 = np.mean(np.abs(X - decoded))
        effects += [L1, L1, L1]
    
    return effects



## Movie creation

cosmos = mda.Universe("/wrk/eurastof/binding_spots_project/gpcr_sampling/b2ar/b2ar_centered_aligned/popc/step6.6_equilibration.gro",
                      "/wrk/eurastof/binding_spots_project/gpcr_sampling/b2ar/b2ar_centered_aligned/popc/centered_aligned10.xtc")
with open("/wrk/eurastof/binding_spots_project/HFSP---Lipid-binding-states/calculations/b2ar_common.ndx") as f:
        lines = "".join(f.readlines())
index = " ".join(re.findall(r"\d+", lines)[1:])
common_ca = cosmos.select_atoms(f"bynum {index}")
common_ca.write("common_ca.gro")

enc_vals = enc[:,1]
n_bins = 10

bins = np.linspace(np.min(enc_vals), np.max(enc_vals), n_bins)

write_file = "enc_dim_2.xtc"


with mda.Writer(write_file, common_ca.n_atoms) as W:
    for i in range(bins.shape[0] - 1):
        
        in_bin = np.where((enc_vals > bins[i]) & (enc_vals < bins[i + 1]))[0]
        
        frames_in_bin = np.array(X[in_bin,:])
        avg_frame_in_bin = np.mean(frames_in_bin, axis=0).reshape(1, frames_in_bin.shape[1])
        distances_to_avg = np.mean(np.sqrt((frames_in_bin - avg_frame_in_bin)**2), axis=1)
        closest = np.argmin(distances_to_avg)
        closest_coords = frames_in_bin[closest,:].reshape(int(frames_in_bin.shape[1]/3), 3)
        common_ca.positions = closest_coords
        W.write(common_ca)
    
    
    

